In [ ]:
!pip uninstall -y kaggle
!pip install kaggle==1.5.6
%env KAGGLE_USERNAME = #########
%env KAGGLE_KEY = ###########################
!kaggle competitions download -c fcis-sc-deeplearning-competition

In [ ]:
!unzip "/content/fcis-sc-deeplearning-competition.zip"

In [ ]:
import numpy as np
import pandas as pd
import glob
import os
from tqdm import tqdm
from random import shuffle
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras import models ,layers ,optimizers
from keras.layers import Dense, Input
from os import listdir
from sklearn.model_selection import train_test_split
from keras.applications import vgg19
from keras.models import Model, Sequential 
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical, np_utils


# -------------------------------------------------------------
# create labels ...
def create_label(file_name):
  if file_name == 'airport_inside':
    label = 1
  elif file_name == 'bakery':
    label = 2
  elif file_name == 'bedroom':
    label = 3
  elif file_name == 'greenhouse':
    label = 4
  elif file_name == 'gym':
    label = 5
  elif file_name == 'kitchen':
    label = 6
  elif file_name == 'operating_room':
    label = 7
  elif file_name == 'poolinside':
    label = 8
  elif file_name == 'restaurant':
    label = 9
  elif file_name == 'toystore':
    label = 10
  return label

# -------------------------------------------------------------
# Read train Data ...
training_data = []
label = []
Img_Size = (224, 224)
train_folder = '/content/train/train'
for filename in os.listdir(train_folder):
  path = os.path.join(train_folder, filename)
  for img in tqdm(os.listdir(path)):
    img_path = os.path.join(path, img)
    imgs = img_to_array(image.load_img(img_path, target_size=(Img_Size)))
    imgs = np.expand_dims(imgs, axis=0)
    imgs = preprocess_input(imgs)
    training_data.append(imgs)
    label.append(create_label(filename))

training_data = np.array(training_data)    
label = np.array(label)    

training_data=np.rollaxis(training_data,1,0)
training_data=training_data[0]
# -------------------------------------------------------------

# e_label = np_utils.to_categorical(label, classes)

num_classes = 10

encoder = LabelEncoder()
encoder.fit(label)
e_label = encoder.transform(label)

Y = np_utils.to_categorical(e_label, num_classes)

x_train, x_val, y_train, y_val = train_test_split(training_data, Y, test_size=0.2,
                                                    shuffle=True, stratify= Y)


img_shape = (224, 224, 3)
 
# Image Augmentation ...
# train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=50,
#                                    width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
#                                    horizontal_flip=True, fill_mode='nearest')

# val_datagen = ImageDataGenerator(rescale=1./255)
# train_generator = train_datagen.flow(x_train, y_train, batch_size=50)
# val_generator = val_datagen.flow(x_val, y_val, batch_size=30)

# # New Model ...
# -------------------------------------------------------------

# VGG-19 Model ...
image_input = Input(shape=(224, 224, 3))

model = vgg19.VGG19(input_tensor=image_input, include_top=True,weights='imagenet')
model.summary()

out = Dense(num_classes, activation='softmax', name='output')(model.layers[-2].output)
vgg_model = Model(image_input, out)
for layer in vgg_model.layers[:-1]:
	layer.trainable = False
vgg_model.summary()
# Adam(lr=1e-5)

vgg_model.compile(loss='categorical_crossentropy',optimizer=optimizers.RMSprop(lr=1e-4),metrics=['accuracy'])
history = vgg_model.fit(x_train, y_train, batch_size=32,
                        validation_data=(x_val,y_val),
                        epochs=25, verbose=1)
                        
# history = vgg_model.fit(training_data, Y, batch_size=32,
                        #  epochs=200, verbose=1)
(loss, accuracy) = vgg_model.evaluate(x_val, y_val, batch_size=10, verbose=1)

print("Loss ={:.4f} ". format(loss))
print("Accuracy = {:.2f}%".format(accuracy*100))
model.save("final model.h5")
# ----------------------------------------------------------

# Read test data ...
test_folder= '/content/test/test/'
testing_data = []
img_id = []
for img in tqdm(os.listdir(test_folder)):
  img_path = os.path.join(test_folder, img)
  imgs = img_to_array(image.load_img(img_path, target_size=(Img_Size)))
  imgs = np.expand_dims(imgs, axis=0)
  imgs = preprocess_input(imgs)
  testing_data.append(imgs)
  img_id.append(img)

testing_data = np.array(testing_data)    
img_id = np.array(img_id)    

testing_data=np.rollaxis(testing_data,1,0)
testing_data=testing_data[0]
# ---------------------------------------------------------------------

predict_class = vgg_model.predict(testing_data,  verbose=0)
# decode label ...
pred_classes = np.argmax(predict_class, axis=-1)
class_label = encoder.inverse_transform(pred_classes)

# Save in Submit file ...
submit_file= []
for i in range(predict_class.shape[0]):
  submit_file.append([img_id[i], class_label[i]])

submit = pd.DataFrame(submit_file, columns=['Id', 'Label'])  
submit.to_csv("submit.csv",index=False)



100%|██████████| 277/277 [00:02<00:00, 124.32it/s]


Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     